# Giới thiệu

Trong bài này, chúng ta học về cách tổng hợp thông tin từ `DataFrame`.

Về dữ liệu, chúng ta sẽ sử dụng lại dữ liệu về các tướng trong Liên Minh.


In [1]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/Levytan/MIS.2019/master/Data/lol_champions.csv', sep = ';', decimal = ',')
data = data.assign(release_date = pd.to_datetime(data.release_date))

data.head()

,champion,title,release_date,attack_at_lv1,attack_at_lv18,health_at_lv1,health_at_lv18,primary
0,Aatrox,the Darkin Blade,2013-06-13,60.0000,145.0,580.00,2110.0,Juggernaut
1,Ahri,the Nine-Tailed Fox,2011-12-14,53.0400,104.0,526.00,2090.0,Burst
2,Akali,the Rogue Assassin,2010-05-11,62.4000,118.5,575.00,2190.0,Assassin
3,Alistar,the Minotaur,2009-02-21,61.1116,122.7,573.36,2375.4,Vanguard
4,Amumu,the Sad Mummy,2009-06-26,53.3800,118.0,613.12,2041.1,Vanguard


# Tổng hợp thông tin căn bản

## Dùng phương thức có sẵn

Như đã học trong `Series`, chúng ta có một số phương thức tổng hợp thông tin như :
- `.mean()`, `.median()` : trả về giá trị trung bình, trung vị.
- `.min()`, `.max()` : trả về giá trị lớn nhất, nhỏ nhất.
- `.std()`, `.var()` : trả về độ lệch chuẩn, phương sai.
- `.sum()`, `.prod()` : trả về tổng, tích các phần tử.
- `.describe()` : trả về thống kê mô tả.

In [0]:
s = pd.Series([1, 2, 3])

s.describe()

count    3.0
mean     2.0
std      1.0
min      1.0
25%      1.5
50%      2.0
75%      2.5
max      3.0
dtype: float64

**Câu hỏi** : Nếu trong `Series` có giá trị `NaN`, các phương thức trên sẽ hoạt động như thế nào?

Đối với `DataFrame`, chúng ta cũng có những phương thức tương tự. 

Mặc định, các phương thức trên sẽ tổng hợp thông tin theo cột.

In [0]:
data.mean()

attack_at_lv1       59.351257
attack_at_lv18     114.291781
health_at_lv1      556.412603
health_at_lv18    2059.282877
dtype: float64

In [2]:
data.max(numeric_only = True)

attack_at_lv1       70.00
attack_at_lv18     153.00
health_at_lv1      625.64
health_at_lv18    2466.00
dtype: float64

In [0]:
data.describe()

,attack_at_lv1,attack_at_lv18,health_at_lv1,health_at_lv18
count,146.000000,146.000000,146.000000,146.000000
mean,59.351257,114.291781,556.412603,2059.282877
std,6.204104,14.653974,35.273908,135.510481
min,40.368000,50.000000,400.000000,1420.000000
25%,54.953425,105.850000,533.100000,2013.500000
50%,60.000000,114.250000,563.240000,2055.100000
75%,64.000000,121.725000,582.180000,2123.000000
max,70.000000,153.000000,625.640000,2466.000000


Ngoài ra, bạn có thể tổng hợp thông tin theo dòng bằng cách chỉ ra `axis = 'columns'` trong các phương thức trên.

In [0]:
df = pd.DataFrame([
    [1, 2, 3],
    [4, 5, 6]
], columns = list('ABC'))

df

,A,B,C
0,1,2,3
1,4,5,6


In [0]:
df.max(axis = 'columns')

0    3
1    6
dtype: int64

In [0]:
df.mean(axis = 'columns')

0    2.0
1    5.0
dtype: float64

## Dùng các hàm tổng hợp tự định nghĩa
Ngoài cách dùng các phương thức có sẵn, bạn có thể dùng các hàm tổng hợp thông tin do chính bạn tự định nghĩa bằng cách dùng phương thức `.apply()` bằng cú pháp :
```
<tên_DataFrame>.apply(<hàm_định_nghĩa>)
```


In [5]:
def get_year(x):
    return x.year

data.release_date.apply(get_year)

0      2013
1      2011
2      2010
3      2009
4      2009
5      2009
6      2009
7      2009
8      2016
9      2014
10     2015
11     2009
12     2011
13     2014
14     2011
15     2016
16     2010
17     2009
18     2009
19     2012
20     2012
21     2009
22     2012
23     2015
24     2012
25     2009
26     2010
27     2009
28     2012
29     2011
       ... 
116    2013
117    2009
118    2010
119    2009
120    2009
121    2009
122    2009
123    2010
124    2012
125    2011
126    2009
127    2014
128    2012
129    2011
130    2010
131    2011
132    2009
133    2011
134    2017
135    2011
136    2010
137    2013
138    2011
139    2019
140    2013
141    2012
142    2012
143    2009
144    2017
145    2012
Name: release_date, Length: 146, dtype: int64

Các phương pháp tổng hợp thông tin trên cho thấy cái nhìn tổng quát về dữ liệu. 

Tuy nhiên, bao nhiêu đó là chưa đủ nếu bạn muốn tổng hợp thông tin về từng phân loại tướng. Vì vậy, chúng ta cùng tìm hiểu đến các phương pháp tổng hợp nâng cao.

# Tổng hợp thông tin nâng cao : Gom nhóm (Group by)

Bạn có thể hiểu Group by là một quá trình gồm 3 bước : tách ra (split) - tính toán (apply) - tập hợp lại (combine).

Dưới đây là minh họa cho Group by (trong đó việc tính toán là tính tổng)

![Fig 1](https://jakevdp.github.io/PythonDataScienceHandbook/figures/03.08-split-apply-combine.png)

## Split

Tách dữ liệu là hành động chia `DataFrame` thành nhiều `DataFrame` khác dựa trên giá trị của `cột_tách`. Để tách dữ liệu, ban dùng cú pháp sau :
```
<tên_DataFrame>.groupby(<cột_tách>)
```

In [0]:
# tách data theo primary
data.groupby('primary')

Chú ý, kết quả trả về không phải là một danh sách các `DataFrame` được tách ra từ `DataFrame` gốc mà là một đối tượng `GroupBy`. Bạn có thể lặp qua đối tượng `GroupBy` giống như ví dụ sau

In [0]:
for primary, df in data.groupby('primary') :
    print(f'Phân loại {primary} có {df.shape[0]} tướng.')

Phân loại Artillery có 4 tướng.
Phân loại Assassin có 15 tướng.
Phân loại Battlemage có 11 tướng.
Phân loại Burst có 14 tướng.
Phân loại Catcher có 7 tướng.
Phân loại Diver có 16 tướng.
Phân loại Enchanter có 7 tướng.
Phân loại Juggernaut có 13 tướng.
Phân loại Marksman có 20 tướng.
Phân loại Skirmisher có 7 tướng.
Phân loại Specialist có 14 tướng.
Phân loại Vanguard có 13 tướng.
Phân loại Warden có 5 tướng.


## Apply

Khi đã có đối tượng `GroupBy`, bạn có thể gọi ra các phương thức tổng hợp cơ bản ở trên.

In [0]:
# in ra lượng máu nhiều nhất ở cấp 1 của từng phân loại
data.groupby('primary')['health_at_lv1'].max()

primary
Artillery     560.00
Assassin      602.00
Battlemage    589.00
Burst         585.00
Catcher       585.00
Diver         601.28
Enchanter     575.00
Juggernaut    620.00
Marksman      582.00
Skirmisher    625.64
Specialist    610.00
Vanguard      615.00
Warden        600.00
Name: health_at_lv1, dtype: float64

In [0]:
# in ra lượng máu trung bình ở cấp 18 của từng phân loại
data.groupby('primary')['health_at_lv18'].mean()

primary
Artillery     2074.000000
Assassin      2114.386667
Battlemage    2061.136364
Burst         2081.478571
Catcher       2078.442857
Diver         2045.993750
Enchanter     1805.957143
Juggernaut    2159.584615
Marksman      2024.550000
Skirmisher    2077.071429
Specialist    1968.757143
Vanguard      2107.376923
Warden        2168.000000
Name: health_at_lv18, dtype: float64